In [1]:
from sklearn.datasets import fetch_covtype
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# 1. 데이터 
datasets = (fetch_covtype())

x = pd.DataFrame(datasets["data"])
y = (datasets["target"])




In [2]:

# y를 ndarry 로 변환해줌
# 또는 y = y.values 사용가능
# y = y.to_numpy()


In [3]:
from sklearn.preprocessing import OneHotEncoder
# 원핫 인코더를 적용하기전 쉐이프 변환
print(y.shape)
y = y.reshape(581012, 1)
print(y.shape)

# 원핫 인코더 변환
ohe = OneHotEncoder()
ohe.fit(y)
y = ohe.transform(y)

# 넘파이 변환
y= y.toarray()

(581012,)
(581012, 1)


In [4]:
print(x.shape,y.shape) 

print(np.unique(y)) # [1 2 3 4 5 6 7]
# 불균형확인
# 참고로 이 y 값은 0 클래스가 없기 대문에 0 을 지워줘야합니다 ... ㅂㄷㅂㄷ

print(np.unique(y,return_counts=True)) # [211840, 283301,  35754,   2747,   9493,  17367,  20510




(581012, 54) (581012, 7)
[0. 1.]
(array([0., 1.]), array([3486072,  581012], dtype=int64))


In [5]:
# print(x.info())
# print(x.describe())
# 데이터 분리
x_train, x_test, y_train,y_test = train_test_split(
                                                    x,
                                                    y,
                                                    train_size=0.8,
                                                    shuffle = True,
                                                    #  stratify 는 데이터 불균형을 해결해줌
                                                    stratify=y,
                                                    random_state=21
                                                   )
# print(x.columns)
# y 를 원 핫 인코딩 변환

print(y_train[:10])
print("y_train shape : ",y_train.shape)
print("y_test shape : ",y_test.shape)

[[1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]]
y_train shape :  (464809, 7)
y_test shape :  (116203, 7)


In [6]:
model = Sequential([
    Dense(128,activation="relu",input_shape=(54,)),
    Dense(128,activation="relu") ,
    Dense(128,activation="relu") ,
    Dense(64,activation="relu") ,
    Dense(64,activation="relu") ,
    Dense(32,activation="relu") ,
    Dense(32,activation="relu") ,
    # 다중분류모델의 활성화 함수는 softmax 입니다
    Dense(7,activation="softmax")
])

In [7]:
# 이진분류는 마지막 활성함수는 sigmoid + loss 는 바이너리 크로스 엔트로피 
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
monitor='val_loss',
patience=10, 
verbose=1, 
mode='min')

# 훈련값이 int 형이기 때문에 sparse 를 사용합니다.
model.compile(loss="categorical_crossentropy"
              ,optimizer="adam"
              ,metrics=["accuracy"]
              )

hist = model.fit(x_train, y_train, epochs=10, 
                 validation_split=0.2,batch_size=256,
                 callbacks = [early_stopping])

# metrics 에 accuracy 사용가능


Epoch 1/10
1453/1453 [==============================] - 10s 6ms/step - loss: 1.0270 - accuracy: 0.6075 - val_loss: 0.7145 - val_accuracy: 0.6871
Epoch 2/10
1453/1453 [==============================] - 8s 5ms/step - loss: 0.6663 - accuracy: 0.7128 - val_loss: 0.6561 - val_accuracy: 0.7082
Epoch 3/10
1453/1453 [==============================] - 6s 4ms/step - loss: 0.6111 - accuracy: 0.7361 - val_loss: 0.6989 - val_accuracy: 0.6924
Epoch 4/10
1453/1453 [==============================] - 6s 4ms/step - loss: 0.5796 - accuracy: 0.7498 - val_loss: 0.5691 - val_accuracy: 0.7534
Epoch 5/10
1453/1453 [==============================] - 6s 4ms/step - loss: 0.5461 - accuracy: 0.7653 - val_loss: 0.5207 - val_accuracy: 0.7790
Epoch 6/10
1453/1453 [==============================] - 6s 4ms/step - loss: 0.5211 - accuracy: 0.7757 - val_loss: 0.4833 - val_accuracy: 0.7933
Epoch 7/10
1453/1453 [==============================] - 6s 4ms/step - loss: 0.4933 - accuracy: 0.7876 - val_loss: 0.4677 - val_accuracy

In [8]:
loss,accuracy = model.evaluate(x_test,y_test)

print("loss:",loss,"\nacc :" ,accuracy)


3632/3632 [==============================] - 8s 2ms/step - loss: 0.4324 - accuracy: 0.8114
loss: 0.4324304759502411 
acc : 0.8114420175552368


In [9]:
from sklearn.metrics import accuracy_score

y_predict = model.predict(x_test)
print(y_predict)
y_predict = np.argmax(y_predict,axis=1)


[[3.95993978e-01 6.03622377e-01 1.77406343e-06 ... 1.69859886e-05
  2.36483470e-06 3.62557330e-04]
 [8.19762200e-02 9.17836726e-01 1.42189080e-07 ... 1.67475257e-04
  1.19558985e-08 1.94928962e-05]
 [1.29197042e-10 2.49394542e-03 9.96885836e-01 ... 3.90453758e-09
  4.85122669e-04 1.31177949e-04]
 ...
 [1.10050372e-04 7.36407638e-02 3.14888212e-07 ... 9.26248848e-01
  6.45654596e-09 9.59842872e-10]
 [3.16856414e-01 6.82992935e-01 1.28140243e-09 ... 2.88575954e-08
  2.62833338e-10 1.50558015e-04]
 [7.97117293e-01 2.02872649e-01 7.46851697e-07 ... 8.51367815e-07
  1.19572420e-07 8.37767038e-06]]


In [10]:
print(y_predict)

y_predict = tf.one_hot(y_predict,depth =7)


[1 1 2 ... 4 1 0]


In [11]:
# 데이터 출력

print("예측데이터 : ",y_predict[:20])
print("실제데이터 : ",y_test[:20])

print
print("========================================")
acc = accuracy_score(y_test,y_predict)
print(acc)


예측데이터 :  tf.Tensor(
[[0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]], shape=(20, 7), dtype=float32)
실제데이터 :  [[0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0

In [12]:
model.save("./_data\\"+"fetch(val_acc=0.89).h5")